# Prepare dataset
The purpose of this file is to normalize dataset to have same size along the whole dataset in one run. 
This code finds all files from the directory and cuts each of them into smaller pieces of same *length*, optionally with a *stride*. If the  piece of a file is shorter than a *length* but still longer than a *minimum length treshold* zeroes at the end of a file are padded (to make the size equal to *length*).

To-do list

**Part 1**
* [x] find all files in the directory
* [x] prepare a strategy how to cut files: user gives file length in seconds, but we oparate on array indexes
* [ ] cut sound into pieces 
* [ ] if last file is smaller put zeroes at the end
* [ ] any preprocessing/normalization ?
* [ ] save files

**Part 2**
* [ ] Prepare test, validation and training sets with k-fold validation

## Create list of all files to cut

In [6]:
# find all of the files in the directory
import os

flist=[]
for root, dirs, files in os.walk("../data/xeno-canto-dataset/"):
    for file in files:
        if file.endswith(".mp3"):
            flist.append(os.path.join(root, file))
                
print('Found ', len(flist),' files.')

Found  1782  files.


##  Load example file

In [7]:
import librosa
import matplotlib.pyplot as plt

filePath = "../data/xeno-canto-dataset/Acrocephalus arundinaceus cnt germany type song\Acrocephalus246389.mp3"
signal, sr = librosa.load(filePath) # sr = sampling rate


## Find strategy to cut the file
We want to cut files to smaller pieces of the desired size (5 seconds in example), and stride of 1 second. Stride tells us how different pieces of files will overlap at each other.

In [29]:
size = {'desired': 5, 'minimum': 3, 'stride' : 1} # stride should not be bigger than desired length
endTime = 0

for startTime in range(0,int(len(signal)/sr),size['desired']-size['stride']):
    startTime = startTime 
    endTime = startTime + size['desired']
    start = int(startTime*sr)
    end = int(endTime*sr)
    print('New file...',start/sr,' - ',end/sr)
    print('Start: ',start,'end: ', end, 'length: ', end-start)
    



New file... 0.0  -  5.0
Start:  0 end:  110250 length:  110250
New file... 4.0  -  9.0
Start:  88200 end:  198450 length:  110250
New file... 8.0  -  13.0
Start:  176400 end:  286650 length:  110250
New file... 12.0  -  17.0
Start:  264600 end:  374850 length:  110250
New file... 16.0  -  21.0
Start:  352800 end:  463050 length:  110250
